# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j

# For maps
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 7.5MB/s 


Define the environment (Java & Spark homes)

---

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Starting Spark Session and print the version


---


In [ ]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4500") \
        .getOrCreate()

spark.version

'3.0.1'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!sleep 2
get_ipython().system_raw('./ngrok http 4500 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-11-25 12:51:21--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.198.20.103, 35.174.46.144, 54.227.160.247, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.198.20.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  15.9MB/s    in 0.8s    

2020-11-25 12:51:22 (15.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://767ee60b1595.ngrok.io


# Descargar Datasets

In [ ]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/species.csv -P /dataset
!ls /dataset

bank.csv  characters.csv  planets.csv  species.csv  vehicles.csv


# Windows Partitioning

---



## Ejemplo 1

In [ ]:
!head /dataset/bank.csv

"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"
30;"unemployed";"married";"primary";"no";1787;"no";"no";"cellular";19;"oct";79;1;-1;0;"unknown";"no"
33;"services";"married";"secondary";"no";4789;"yes";"yes";"cellular";11;"may";220;1;339;4;"failure";"no"
35;"management";"single";"tertiary";"no";1350;"yes";"no";"cellular";16;"apr";185;1;330;1;"failure";"no"
30;"management";"married";"tertiary";"no";1476;"yes";"yes";"unknown";3;"jun";199;4;-1;0;"unknown";"no"
59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"
35;"management";"single";"tertiary";"no";747;"no";"no";"cellular";23;"feb";141;2;176;3;"failure";"no"
36;"self-employed";"married";"tertiary";"no";307;"yes";"no";"cellular";14;"may";341;1;330;2;"other";"no"
39;"technician";"married";"secondary";"no";147;"yes";"no";"cellular";6;"may";151;2;-1;0;"unknown";"no"
41;"entrepreneur";"marrie

Leyendo Datos del fichero bank.csv a un Dataframe

In [ ]:
from pyspark.sql.functions import *

bank_df = spark.read.format("csv") \
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/bank.csv")

In [ ]:
bank_df.show()

+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|   unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|     services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|   management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30|   management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|  blue-collar|married|secondary|     no|      0|    yes|  no| unknown| 

**Obtén el balance de las dos personas más jóvenes por tipo de trabajo**


In [ ]:
from pyspark.sql.window import Window

byJob = Window.partitionBy("job").orderBy("age")

bank_df \
  .withColumn("new_column_job", row_number().over(byJob)) \
  .filter(col("new_column_job") <= 2) \
  .select("age", "job", "balance") \
  .orderBy("job", "age") \
  .show()

+---+-------------+-------+
|age|          job|balance|
+---+-------------+-------+
| 22|       admin.|   4111|
| 23|       admin.|      5|
| 23|  blue-collar|    817|
| 23|  blue-collar|   8627|
| 23| entrepreneur|      4|
| 25| entrepreneur|  16874|
| 26|    housemaid|    543|
| 26|    housemaid|   -759|
| 23|   management|    736|
| 24|   management|    172|
| 24|      retired|    366|
| 35|      retired|    285|
| 25|self-employed|    453|
| 26|self-employed|    211|
| 21|     services|    361|
| 21|     services|   1903|
| 19|      student|    103|
| 19|      student|      0|
| 22|   technician|    333|
| 23|   technician|    598|
+---+-------------+-------+
only showing top 20 rows



## Ejercicio 1

**A partir del Dataframe formado a partir del fichero "bank.csv". 
Obtén el Top 3 de máximos balances por estado civil**


---




In [ ]:
byMarital = Window.partitionBy("marital").orderBy(col("balance").desc())

bank_df \
  .withColumn("ranking", row_number().over(byMarital)) \
  .filter(col("ranking") <= 3) \
  .show()

+---+-------------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-------+
|age|          job| marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|ranking|
+---+-------------+--------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-------+
| 54|   management|divorced| tertiary|     no|  26306|    yes|  no|cellular| 11|  feb|      27|       1|   84|       3| failure| no|      1|
| 34|   management|divorced| tertiary|     no|  13204|    yes| yes|cellular| 20|  nov|     197|       2|   -1|       0| unknown| no|      2|
| 51|self-employed|divorced|  primary|     no|  10924|     no|  no|cellular|  6|  may|     106|       2|   -1|       0| unknown| no|      3|
| 60|      retired| married|  primary|     no|  71188|     no|  no|cellular|  6|  oct|     205|       1|   -1|       0| unknown| no|      1|
| 42| entrepr

## Ejercicio 2



**Carga el fichero de vehicles.csv en un DataFrame**

In [ ]:
!head /dataset/vehicles.csv

name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class
Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled
T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft
X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,NA,repulsorcraft
TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NA,6.4,1200,1,0,65,2 days,starfighter
Snowspeeder,t-47 airspeeder,Incom corporation,NA,4.5,650,2,0,10,none,airspeeder
TIE bomber,TIE/sa bomber,Sienar Fleet Systems,NA,7.8,850,1,0,none,2 days,space/planetary bomber
AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,20,60,5,40,1000,NA,assault walker
AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Military Research",NA,2,90,2,0,200,none,walker
Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin

In [ ]:
vehiclesDF_all = spark.read.format("csv")\
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/vehicles.csv")


In [ ]:
vehiclesDF_all.show(truncate=False)

+------------------------------+-------------------------------+----------------------------------------------------------+---------------+------+----------------------+----+----------+--------------+---------------+-------------------------+
|name                          |model                          |manufacturer                                              |cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|consumables    |vehicle_class            |
+------------------------------+-------------------------------+----------------------------------------------------------+---------------+------+----------------------+----+----------+--------------+---------------+-------------------------+
|Sand Crawler                  |Digger Crawler                 |Corellia Mining Corporation                               |150000         |36.8  |30                    |46  |30        |50000         |2 months       |wheeled                  |
|T-16 skyhopper             

**Para cada uno de los vehículos, obtén la diferencia de precio (*cost_in_credits*) para cada producto con respecto al más barato en la misma clase de vehículo**


---



In [ ]:
from pyspark.sql.types import IntegerType

vehicles_filtered = vehiclesDF_all \
  .filter(col("cost_in_credits") != "NA") \
  .withColumn("cost_in_credits", col("cost_in_credits").cast(IntegerType()))
vehicles_filtered.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- cost_in_credits: integer (nullable = true)
 |-- length: string (nullable = true)
 |-- max_atmosphering_speed: string (nullable = true)
 |-- crew: integer (nullable = true)
 |-- passengers: string (nullable = true)
 |-- cargo_capacity: string (nullable = true)
 |-- consumables: string (nullable = true)
 |-- vehicle_class: string (nullable = true)



In [ ]:
from pyspark.sql.functions import *

byClass = Window.partitionBy("vehicle_class") \
  .orderBy(col("cost_in_credits").asc())


vehicles_filtered \
  .withColumn("cost_difference", min(col("cost_in_credits")).over(byClass) - col("cost_in_credits")) \
  .withColumn("min_cost", min(col("cost_in_credits")).over(byClass)) \
  .orderBy(col("vehicle_class"), col("cost_in_credits").desc()) \
  .show()

+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+---------------+--------+
|                name|               model|        manufacturer|cost_in_credits|length|max_atmosphering_speed|crew|passengers|cargo_capacity|    consumables|       vehicle_class|cost_difference|min_cost|
+--------------------+--------------------+--------------------+---------------+------+----------------------+----+----------+--------------+---------------+--------------------+---------------+--------+
|Raddaugh Gnasp fl...|Raddaugh Gnasp fl...|Appazanna Enginee...|          14750|     7|                   310|   2|         0|            20|           none|         air speeder|              0|   14750|
|       Droid gunship|   HMP droid gunship|Baktoid Fleet Ord...|          60000|  12.3|                   820|   0|         0|             0|           none|          airspeeder|      

# Joins

## Ejercicio 3

**Crea los dataframes correspondientes para los ficheros "characters.csv" y "planets.csv". <br/>
Obtén la gravedad del planeta para cada personaje. Selecciona sólo el nombre del personaje y planeta además de su gravedad**


---




In [ ]:
charactersDF_all = spark.read.format("csv")\
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/characters.csv")

In [ ]:
charactersDF_all.show(truncate=False)

+---------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|name                 |height|mass |hair_color   |skin_color      |eye_color|birth_year|gender       |homeworld |species       |
+---------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|Luke Skywalker       |172   |77   |blond        |fair            |blue     |19BBY     |male         |Tatooine  |Human         |
|C-3PO                |167   |75   |NA           |gold            |yellow   |112BBY    |NA           |Tatooine  |Droid         |
|R2-D2                |96    |32   |NA           |white, blue     |red      |33BBY     |NA           |Naboo     |Droid         |
|Darth Vader          |202   |136  |none         |white           |yellow   |41.9BBY   |male         |Tatooine  |Human         |
|Leia Organa          |150   |49   |brown        |light           |brown    |19BBY     |female   

In [ ]:
planetsDF_all = spark.read.format("csv")\
  .option("sep", ";") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/planets.csv")

In [ ]:
planetsDF_all.show(truncate=False)

+--------------+---------------+--------------+--------+----------------------+--------------------------------------+----------------------------------------+-------------+-----------+
|name          |rotation_period|orbital_period|diameter|climate               |gravity                               |terrain                                 |surface_water|population |
+--------------+---------------+--------------+--------+----------------------+--------------------------------------+----------------------------------------+-------------+-----------+
|Alderaan      |24             |364           |12500   |temperate             |1 standard                            |grasslands, mountains                   |40           |2000000000 |
|Yavin IV      |24             |4818          |10200   |temperate, tropical   |1 standard                            |jungle, rainforests                     |8            |1000       |
|Hoth          |23             |549           |7200    |frozen        

In [ ]:
from pyspark.sql.functions import *

charactersDF_all \
  .join(planetsDF_all, charactersDF_all["homeworld"] == planetsDF_all["name"]) \
  .select(charactersDF_all["name"], planetsDF_all["name"], col("gravity")) \
  .show()

+--------------------+-----------+-------------+
|                name|       name|      gravity|
+--------------------+-----------+-------------+
|          Wat Tambor|      Skako|            1|
|      Wilhuff Tarkin|     Eriadu|   1 standard|
|          Tion Medon|     Utapau|   1 standard|
|   Poggle the Lesser|   Geonosis| 0.9 standard|
|       Ratts Tyerell|Aleen Minor|           NA|
|               Dooku|    Serenno|           NA|
|            Shaak Ti|      Shili|            1|
|           Sly Moore|     Umbara|           NA|
|          Mon Mothma|  Chandrila|            1|
|           Chewbacca|   Kashyyyk|   1 standard|
|             Tarfful|   Kashyyyk|   1 standard|
|Wicket Systri War...|      Endor|0.85 standard|
|                Yoda|         NA|           NA|
|               IG-88|         NA|           NA|
|        Arvel Crynyd|         NA|           NA|
|        Qui-Gon Jinn|         NA|           NA|
|              R4-P17|         NA|           NA|
|                Fin

## Ejercicio 4

**Revisa el plan de ejecución del ejercicio 3. ¿Qué tipo de join se está ejecutando? ¿Por qué?**

---

**Después de revisar el plan de ejecución, ejecuta las siguientes instrucciones**

---

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", '0')

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

'0'

**Vuelve a ejecutar la consulta del ejercicio 3 que contiene el Join**

---

## Ejercicio 5

**Crea un DataFrame a partir del fichero de "species.csv" y reparticiona este y el DataFrame de Characters a 100 particiones**


---



In [ ]:
charactersDF_repartition = charactersDF_all.repartition(100)

In [ ]:
speciesDF_all = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load("/dataset/species.csv")

In [ ]:
speciesDF_all.show()

+--------------+--------------+-----------+--------------+--------------------+--------------------+--------------------+----------------+--------------+--------------+
|          name|classification|designation|average_height|         skin_colors|         hair_colors|          eye_colors|average_lifespan|      language|     homeworld|
+--------------+--------------+-----------+--------------+--------------------+--------------------+--------------------+----------------+--------------+--------------+
|          Hutt|     gastropod|   sentient|           300|   green, brown, tan|                  NA|         yellow, red|            1000|       Huttese|     Nal Hutta|
|Yoda's species|        mammal|   sentient|            66|       green, yellow|        brown, white|brown, green, yellow|             900|Galactic basic|            NA|
|    Trandoshan|       reptile|   sentient|           200|        brown, green|                none|      yellow, orange|              NA|          Dosh|  

In [ ]:
speciesDF_repartition = speciesDF_all.repartition(100)

## Ejercicio 6

**Obtén la clasificación de especies para cada personaje. Selecciona sólo el nombre del personaje y su clasificación de especie**<br>
Usa los datframes reparticionados


---



In [ ]:
classDF = charactersDF_repartition \
  .join(speciesDF_repartition, charactersDF_repartition["species"] == speciesDF_repartition["name"]) \
  .select(charactersDF_repartition["name"], speciesDF_repartition["classification"]) \


In [ ]:
classDF.show()

+--------------------+--------------+
|                name|classification|
+--------------------+--------------+
|          Zam Wesell|     reptilian|
|     Dexter Jettster|     amphibian|
|           Nien Nunb|        mammal|
|Wicket Systri War...|        mammal|
|             Sebulba|        mammal|
|                Yoda|        mammal|
|     Luminara Unduli|        mammal|
|       Barriss Offee|        mammal|
|          Wat Tambor|        mammal|
|          Tion Medon|        mammal|
|      Ben Quadinaros|            NA|
|          Adi Gallia|        mammal|
|              Greedo|      sentient|
|          Mas Amedda|     amphibian|
|            Shaak Ti|        mammal|
|             Gasgano|            NA|
|            Plo Koon|            NA|
|         Nute Gunray|            NA|
|          Darth Maul|        mammal|
|           Eeth Koth|        mammal|
+--------------------+--------------+
only showing top 20 rows



## Ejercicio 7

**Ejecuta la siguiente operación sobre el DataFrame del ejercicio 6 y observa la diferencia de reparto de rows entre las particiones**

---



In [ ]:
from pyspark.sql.functions import *

classDF \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("count").desc()) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|        183|   35|
|        170|    5|
|        169|    3|
|         58|    2|
|         65|    2|
|        173|    2|
|          8|    2|
|         43|    2|
|        134|    2|
|         89|    2|
|         51|    1|
|         49|    1|
|        140|    1|
|        176|    1|
|         47|    1|
|        175|    1|
|        127|    1|
|         50|    1|
|        179|    1|
|         64|    1|
+-----------+-----+
only showing top 20 rows



In [ ]:
classDF \
  .repartition(20) \
  .withColumn("partitionId", spark_partition_id()) \
  .groupBy("partitionId") \
  .count() \
  .orderBy(col("count").desc()) \
  .show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|         17|    7|
|          1|    7|
|         18|    7|
|          2|    6|
|         16|    5|
|          4|    5|
|          5|    5|
|         19|    5|
|          3|    5|
|          6|    4|
|          0|    4|
|         15|    3|
|         10|    3|
|          9|    3|
|          7|    3|
|         12|    2|
|         13|    2|
|          8|    2|
|         11|    2|
|         14|    2|
+-----------+-----+

